# Data processing 

Import racing stats and results

In [130]:
import pandas as pd

HrData = pd.read_csv("/Users/leomccarthy/Documents/Projects/Horse_racing/Bet Turtle/MasterStat.csv", delimiter=',', header = 0) 

Remove non-runners 

In [131]:
HrData = HrData.loc[HrData['Status'] == 'Runner',]

HrData = HrData.assign(Win=0)
HrData.loc[HrData['Position'] == 1, 'Win'] = 1

HrData["HWM"].fillna(0,inplace = True)
HrData.dropna(subset=['Position'], inplace = True)

#HrData["gid"] = HrData["Date"] + HrData["Time"]
#HrData['gid'] = le.fit_transform(HrData['gid'])
#HrData.sort_values("gid", inplace = True)

#groups = np.array(HrData.gid.value_counts(sort = False))
#print(HrData[['gid','Course',"Time"]].to_string())



Combine some feature strings to generate a unique Race ID for each race

In [132]:
HrData["RaceID"] = HrData["Date"] + HrData["Course"] + HrData["Time"]
HrData["RaceID"] = HrData["RaceID"].astype('category')

Using some limited domain knowlegde, generate a range of features that attempt to represent the average ability of all the horses in each race. This is a posisble technique to overcome the problem that the absolute ability of a horse is potentialy of far less importance than the relative ability compared to other horses in the race 

In [133]:
races = HrData["RaceID"].cat.categories

In [134]:
for race in races:
    HrData.loc[HrData["RaceID"] == race,"mean_form"] = HrData.loc[HrData["RaceID"] == race,"Form"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_last_ran"] = HrData.loc[HrData["RaceID"] == race,"LastRan"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_last_won"] = HrData.loc[HrData["RaceID"] == race,"LastWon"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_or"] = HrData.loc[HrData["RaceID"] == race,"OR"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_last_or"] = HrData.loc[HrData["RaceID"] == race,"Last_OR"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_hwm"] = HrData.loc[HrData["RaceID"] == race,"HWM"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_speed"] = HrData.loc[HrData["RaceID"] == race,"Speed"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_runs"] = HrData.loc[HrData["RaceID"] == race,"Runs"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_tcstk"] = HrData.loc[HrData["RaceID"] == race,"TCrseStrk"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_vibes"] = HrData.loc[HrData["RaceID"] == race,"Vibes"].mean()
    HrData.loc[HrData["RaceID"] == race,"mode_type"] = HrData.loc[HrData["RaceID"] == race,"Type"].mode()
    HrData.loc[HrData["RaceID"] == race,"mean_prize"] = HrData.loc[HrData["RaceID"] == race,"HTotPrize"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_fitness"] = HrData.loc[HrData["RaceID"] == race,"Fitness"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_tcruns"] = HrData.loc[HrData["RaceID"] == race,"TCrseRun"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_ttstrk"] = HrData.loc[HrData["RaceID"] == race,"TRTypeStrk"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_ttrun"] = HrData.loc[HrData["RaceID"] == race,"TRTypeRun"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_jcstrk"] = HrData.loc[HrData["RaceID"] == race,"JCrseStrk"].mean()
    HrData.loc[HrData["RaceID"] == race,"mean_jcrun"] = HrData.loc[HrData["RaceID"] == race,"JCrseRun"].mean()
    

In [135]:
HrPredictors = HrData.drop(["Date","Course", "Time","Saddle","FormLine","Name","Odds","Trainer","Jockey","Owner",
                           "Sire","Status","RaceComment","Casualty","Travel","Shortlist","Tip","RaceID"
                           ], axis = 1)


HrPredictors = HrPredictors.fillna("NotA")



We now need to encode our categorical features, using label encoder. 

In [136]:
from sklearn import preprocessing

cat_cols = HrPredictors.columns[HrPredictors.dtypes==object].tolist()
le = preprocessing.LabelEncoder()

HrPredictors[cat_cols] = HrPredictors[cat_cols].apply(lambda col: 
                                                      le.fit_transform(col))




In [10]:
#import numpy as np 
#HrPredictors["Pos"] = np.where((HrPredictors["Position"] > 14) & (HrPredictors["Position"] < 25) , 15,HrPredictors["Position"])
#HrPredictors["Pos"] = np.where((HrPredictors["Pos"] > 10) & (HrPredictors["Pos"] < 15) , 10,HrPredictors["Pos"])

## Train - test 

A random train test split is not valid for in this instance, random splits would result in data leakage and a "look-forward" bias. For example, given we want to predict the porbability of a horse winning any given race, if training data included information on that horses perfromance in a future race this would result in issues given that factors such as racing class and racing weight are directly influenced by the performance in the previous race. Thus splits must be done chronologically, we will use the 95 most recent races as a test set.

In [137]:
X_test = HrPredictors.loc[4007:4873,].drop(['Win',"AvgOdds","MinOdds","MaxOdds","DecOdds","Position"], axis = 1)
y_test = HrPredictors.loc[4007:4873,"Win"].values.ravel()

X_train = HrPredictors.loc[:4006,].drop(['Win',"AvgOdds","MinOdds","MaxOdds","DecOdds","Position"], axis = 1)
y_train = HrPredictors.loc[:4006,"Win"].values.ravel()

X_test_prices = HrPredictors.loc[4007:4873,"AvgOdds"]

In [138]:
from sklearn import preprocessing 

x = X_train.values
normalized = preprocessing.Normalizer().fit(x)

x_train_scaled = normalized.transform(X_train)
x_test_scaled = normalized.transform(X_test)

X_train = pd.DataFrame(x_train_scaled, columns = X_train.columns)
X_test = pd.DataFrame(x_test_scaled, columns = X_test.columns)

# Model building 

Import machine learning moules 

In [97]:
from sklearn.model_selection import cross_val_score, cross_val_predict,StratifiedKFold,RepeatedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import auc, accuracy_score, confusion_matrix, precision_score
import xgboost as xgb
from sklearn.svm import SVC
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from xgboost import plot_importance
from xgboost import XGBRanker 
from sklearn.feature_selection import SelectFromModel
from numpy import sort
from boruta import BorutaPy




Coarse feature selection using the boruta algorithm with random forests

In [140]:
boruta_feature_selector = BorutaPy(RandomForestClassifier(), n_estimators='auto', verbose=2, random_state=4242, max_iter = 100, perc = 60)
fs = boruta_feature_selector.fit(X_train.values, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	139
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	18
Tentative: 	56
Rejected: 	65
Iteration: 	9 / 100
Confirmed: 	18
Tentative: 	56
Rejected: 	65
Iteration: 	10 / 100
Confirmed: 	18
Tentative: 	56
Rejected: 	65
Iteration: 	11 / 100
Confirmed: 	18
Tentative: 	56
Rejected: 	65
Iteration: 	12 / 100
Confirmed: 	19
Tentative: 	44
Rejected: 	76
Iteration: 	13 / 100
Confirmed: 	19
Tentative: 	44
Rejected: 	76
Iteration: 	14 / 100
Confirmed: 	19
Tentative: 	44
Rejected: 	76
Iteration: 	15 / 100
Confirmed: 	19
Tentative: 	44
Rejected: 	76
Iteration: 	16 / 100
Confirmed: 	19
Tenta

In [141]:
X_train = X_train.loc[:, fs.support_ == 1]
X_test = X_test.loc[:, fs.support_ == 1]

In [142]:
list(X_train.columns)

['Runners',
 'Type',
 'Form',
 'OddsPos',
 'HTotPrize',
 'TCrseStrk',
 'T60Win',
 'T60Strk',
 'TJWin',
 'TJRun',
 'TJStrk',
 'JRTypeStrk',
 'J14Run',
 'J60Win',
 'OCrseStrk',
 'ORTypeRun',
 'Fitness',
 'Ability',
 'Vibes',
 'Market',
 'Rank',
 'FormRate',
 'mean_last_ran',
 'mean_tcstk',
 'mean_prize',
 'mean_tcruns']

### XGB
Tune and fit xgboost using grid search and 4-fold cross validation 

In [143]:
param_gridXgb = {
        
        'silent': [False],
        'max_depth': [2,3,6,8,10],
        'learning_rate': [0.001, 0.01, 0.05 ,0.1, 0.2],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 2.0, 10.0],
        'n_estimators': [200,600,800], 
        'scale_pos_weight':[3,5] }


clfXgb = xgb.XGBClassifier(objective = 'binary:logistic',nthread = 7, verbose = 0)


Random_searchXgb = RandomizedSearchCV(clfXgb, param_distributions = param_gridXgb, n_iter = 100,scoring = 'f1',cv = 4)

Search_resultXgb = Random_searchXgb.fit(X_train,y_train)



In [144]:
clf_xgb = Search_resultXgb.best_estimator_


In [171]:
xgb_pred = clf_xgb.predict(X_test)

bets = np.where((xgb_pred == 1) & (X_test_prices > 2),1,0)

In [169]:
#preds = np.where((probCompare["preds"] > probCompare["odds_prob"] + 0.05) & (probCompare["preds"] > 0.5)  ,1,0)
#odds = HrPredictors['AvgOdds'] 
#bets = np.where((xgb_probs[:,1] > 0.5),1,0)

confusion_matrix(bets,y_test)

array([[599,  77],
       [ 89,  18]])

In [188]:
PL = sum(X_test_prices[(bets == 1) & (y_test == 1)]) - (bets==1).sum()
SR = len(X_test_prices[(bets == 1) & (y_test == 1)])/(bets==1).sum() *100
ROI = PL/(bets==1).sum() * 100

print('Profit =', PL ,"pts",' Number of bets =',(bets==1).sum(), 'which is',
      round((bets==1).sum()/17), 'per day','and ROI =',round(ROI,2),"%" ,"and SR =",
      round(SR,1),"%", sep = None)

Profit = -37.32000000000001 pts  Number of bets = 122 which is 7.0 per day and ROI = -30.59 % and SR = 19.7 %


### Naive Bayes

In [192]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

nb_pred = gnb.fit(X_train, y_train).predict(X_test)

bets = np.where((nb_pred == 1) & (X_test_prices > 2),1,0)

In [193]:
confusion_matrix(bets,y_test)

array([[647,  87],
       [ 41,   8]])

In [194]:
PL = sum(X_test_prices[(bets == 1) & (y_test == 1)]) - (bets==1).sum()
SR = len(X_test_prices[(bets == 1) & (y_test == 1)])/(bets==1).sum() *100
ROI = PL/(bets==1).sum() * 100

print('Profit =', PL ,"pts",' Number of bets =',(bets==1).sum(), 'which is',
      round((bets==1).sum()/17), 'per day','and ROI =',round(ROI,2),"%" ,"and SR =",
      round(SR,1),"%", sep = None)

Profit = -18.15 pts  Number of bets = 49 which is 3.0 per day and ROI = -37.04 % and SR = 16.3 %


## Stacking models 

In [211]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

estimators = [
    ('xgb', clf_xgb),
    
    ('nb', clf_nb)
]

stack_clf = StackingClassifier(
    estimators=estimators, final_estimator=KNeighborsClassifier())

clf = stack_clf.fit(X_train,y_train)


In [212]:
stack_pred = clf.predict(X_test)
bets = np.where((stack_pred == 1) & (X_test_prices > 2),1,0)

In [213]:
confusion_matrix(bets,y_test)



array([[661,  88],
       [ 27,   7]])

In [214]:
PL = sum(X_test_prices[(bets == 1) & (y_test == 1)]) - (bets==1).sum()
SR = len(X_test_prices[(bets == 1) & (y_test == 1)])/(bets==1).sum() *100
ROI = PL/(bets==1).sum() * 100

print('Profit =', PL ,"pts",' Number of bets =',(bets==1).sum(), 'which is',
      round((bets==1).sum()/17), 'per day','and ROI =',round(ROI,2),"%" ,"and SR =",
      round(SR,1),"%", sep = None)


Profit = -11.5 pts  Number of bets = 34 which is 2.0 per day and ROI = -33.82 % and SR = 20.6 %


This initial attempt which includes an xgboost model, a naive bayes classifier and a stacked ensemmble of the two has resulted in a selction of bets that are un profitable. This is however far from an optimal solution and there are several adjustements that could be made.

First of all this was conducted as a simple binary classification problem which does not take into account only horses from each individual race, the problem would need to be restructured in order to take this into acocunt, perhaps this is a leanring to rank problem, which is where I might look next. Nor yet has any effort been made to compare the winning probability with the available price in order to determine when a given bet may be profitable long term, rather than simply looking for the most likely winners, as market prices are likely to show this already.  